https://pytorch-lightning.readthedocs.io/en/latest/notebooks/lightning_examples/augmentation_kornia.html

In [ ]:
import pandas as pd
import torchvision.io as io
import torch, os
import numpy as np
import pytorch_lightning as pl

train_folder= '../input/petfinder-pawpularity-score/train'
train_paths=  [  os.path.join(train_folder,name)       for name in os.listdir(train_folder)]
train_df=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_df=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
display(train_df)
display(train_df.info())
display(test_df)

In [ ]:
import torchvision
def read_file(path,size=[512,512]):
    img=io.read_image(path)
    resize=torchvision.transforms.Resize(size)
    img=resize(img)
    return img


import matplotlib.pyplot as plt

def display_images(imgs,
                   labels,
                   r,
                   c):
    
    _,axs = plt.subplots(r,c,figsize=(10,10))
    axs=axs.flatten()
    for n,ax in enumerate(axs):
        img=read_file(imgs[n])
        ax.imshow(img.permute(1,2,0))
        ax.set_title( labels[n])
        ax.axis('off')

    plt.tight_layout()
    plt.show()
    
    
display_images(train_paths,train_df.Pawpularity,8,8)

In [ ]:
from torch.utils.data.dataloader import default_collate


In [ ]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,
                 train_files,
                 df,
                 mode,
                 augment_,
                 shuffle,
                 size,
                
                ):
        
    
        self.train_paths= train_files
        self.labels = train_df.Pawpularity
        self.df=df
        self.info=train_df.drop('Pawpularity',axis=1)
        self.augment_ = augment_
        self.shuffle = shuffle
        self.size=size

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        data = self.info.loc[idx]
        img=read_file(self.train_paths[idx],self.size)
        label=self.labels.loc[idx]
        return {'img': img ,
#                 'data':data, 
                'label':label
               }
    
class DataGen(pl.LightningDataModule):
    def __init__(self,
                 ds,
                 train_files,
                 df,
                 augment_,
                 shuffle,
                 size,
                 
                 batch_size
                ):
        
        self.ds=ds
        self.train_files= train_files
        self.labels = train_df.Pawpularity
        self.df=df
        self.info=train_df.drop('Pawpularity',axis=1)
        self.augment_ = augment_
        self.shuffle = shuffle
        self.size=size
        self.batch_size=batch_size
        
        
    def train_dataloader(self):
        ds=self.ds(self.train_files,
                 self.df,
                 'Train',
                 self.augment_,
                 self.shuffle,
                 self.size,
                )
        
        return torch.utils.data.DataLoader(ds,self.batch_size,
                                           num_workers=1,
                                           collate_fn=default_collate
                                          )
    
    def val_dataloader(self):
        ds=self.ds(self.train_files,
                 self.df,
                 'val',
                 self.augment_,
                 self.shuffle,
                 self.size,
                )
        
        return torch.utils.data.DataLoader(ds,self.batch_size,
                                           num_workers=1,
                                           collate_fn=default_collate
                                          )
    
    
ds=DataGen(dataset,
          train_paths,
                 train_df,
                 True,
                 True,
                 (512,512),
                 32
                )

p=ds.train_dataloader()
iter(p).next()
